# Linear Least Squares

In [ ]:
import sys
sys.path.append('lib')

In [ ]:
from typing import List, Tuple
import random
import numpy as np
import pandas as pd

In [ ]:
import nsfg
import brfss
import compstats
import hypothesis
from cdf import Cdf

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from IPython.core.pylabtools import figsize
sns.set_theme()
figsize(11, 5)

load up the NSFG data.

In [ ]:
live = nsfg.read_live_fem_preg().loc[:, [
    'agepreg',
    'totalwgt_lb',
    'finalwgt',
    'birthcat'
]].dropna()

In [ ]:
live.apply(lambda col: col.isna().sum())

The following function computes the intercept and slope of the least squares fit.

In [ ]:
def least_squares(xs: np.array, ys: np.array) -> Tuple[float, float]:
    meanx = xs.mean()
    meany = ys.mean()
    slope = compstats.cov(xs, ys, meanx, meany) / xs.var()
    intercept = meany - (slope * meanx)

    return intercept, slope

Here's the least squares fit to birth weight as a function of mother's age.

In [ ]:
ages = live.agepreg.values
weights = live.totalwgt_lb.values
intercept, slope = least_squares(ages, weights)
print(f'Intercept: {intercept:0.2f}, Slope: {slope:0.4f}')

The intercept is often easier to interpret if we evaluate it at the mean of the independent variable.

In [ ]:
np.round(intercept + slope * ages.mean(), 2)

In this case the mean age is about 25 years and the mean baby weight for a 25 year old mother is 7.3 pounds

And the slope is easier to interpret if we express it in pounds per decade

In [ ]:
np.round(slope * 10, 2)

The slope is 0.27 ounces per year, or 0.17 pounds per decade

The following function evaluates the fitted line at the given `xs`.

In [ ]:
def fit_line(xs, intercept, slope) -> Tuple[np.array, np.array]:
    fit_xs = np.sort(xs)
    fit_ys = intercept + slope * fit_xs
    return fit_xs, fit_ys

Here's a scatterplot of the data with the fitted line.

In [ ]:
p = sns.scatterplot(
    x = ages,
    y = weights,
    color = 'royalblue',
    alpha=0.1,
    s=10
);
p.axline((10, intercept + 10*slope), slope=slope, color='darkred')
p.set(
    xlabel = 'Mothers age (in years)',
    ylabel = 'Birth weight (lbs)',
    ylim = (0, 15)
);

## Residuals

The following functon computes the residuals.

In [ ]:
def residuals(xs: np.array, ys: np.array, intercept: float, slope: float) -> np.array:
    # actual - predicted
    return ys - (intercept + slope * xs)

Now we can add the residuals as a column in the DataFrame.

In [ ]:
live['residual'] = residuals(ages, weights, intercept, slope)

To visualize the residuals, I'll split the respondents into groups by age, then plot the percentiles of the residuals versus the average age in each group.

First I'll make the groups and compute the average age in each group.

In [ ]:
bins = np.arange(10, 48, 3)
indices = np.digitize(live.agepreg, bins)
groups = live.groupby(indices)

age_means = [group.agepreg.mean() for _, group in groups][1:-1]
age_means

Next I'll compute the CDF of the residuals in each group.

In [ ]:
cdfs = [Cdf.from_seq(group.residual) for _, group in groups][1:-1]

The following function plots percentiles of the residuals against the average age in each group.

In [ ]:
def plot_percentiles(age_means: List[float], cdfs: List[Cdf]):
    for percent in [75, 50, 25]:
        weight_percentiles = [cdf.percentile(percent) for cdf in cdfs]
        label = f'{percent}th'
        plt.plot(age_means, weight_percentiles, label=label)

The following figure shows the 25th, 50th, and 75th percentiles.

In [ ]:
plot_percentiles(age_means, cdfs)

plt.xlabel("Mother's age (years)")
plt.ylabel("Residual (lbs)")
plt.xlim([10, 45]);
plt.legend(loc='upper right');

The median is near zero, as expected, and the interquartile range is about 2 pounds. So if we know the mother’s age, we can guess the baby’s weight within a pound, about 50% of the time

Ideally these lines should be flat, indicating that the residuals are random, and parallel, indicating that the variance of the residuals is the same for all age groups. In fact, the lines are close to parallel, so that’s good; but they have some curvature, indicating that the relationship is nonlinear. Nevertheless, the linear fit is a simple model that is probably good enough for some purposes.

## Estimation

The parameters slope and inter are estimates based on a sample; like other estimates, they are vulnerable to sampling bias, measurement error, and sampling error. As discussed in [chaper 8](08_Estimation.ipynb), sampling bias is caused by non-representative sampling, measurement error is caused by errors in collecting and recording data, and sampling error is the result of measuring a sample rather than the entire population.

To assess sampling error, we ask, “If we run this experiment again, how much variability do we expect in the estimates?” We can answer this question by running simulated experiments and computing sampling distributions of the estimates.

To estimate the sampling distribution of `inter` and `slope`, I'll use resampling.

I simulate the experiments by resampling the data; that is, I treat the observed pregnancies as if they were the entire population and draw samples, with replacement, from the observed sample.

In [ ]:
def sampling_distibutions(live: pd.DataFrame, iters=1001) -> Tuple[np.array, np.array]:
    t = []
    for _ in range(iters):
        sample = compstats.resample_rows(live)
        # use the simulated sample to estimate parameters
        t.append(
            least_squares(sample.agepreg, sample.totalwgt_lb)
        )
    inters, slopes = zip(*t)
    # return the estimated intercepts and the estimated slopes
    return np.array(inters), np.array(slopes)

In [ ]:
inters, slopes = sampling_distibutions(live)

Here's an example.

The following function takes a list of estimates and prints the mean, standard error, and 90% confidence interval.

In [ ]:
def summarize(estimates: np.array, actual=None):
    mean = estimates.mean()
    stderr = compstats.std(estimates, mu=actual)
    # 90 percent confidence interval (between 0.05 and 0.95)
    ci = np.percentile(estimates, (5, 95))
    print(f'mean {mean:0.2f}, SE: {stderr:0.3f}, CI: {np.round(ci,2)}')

Here's  the summary for `inter`.

In [ ]:
summarize(inters)

And for `slope`.

In [ ]:
summarize(slopes)

**Exercise:** Use `resample_rows` and generate a list of estimates for the mean birth weight. Use `summarize` to compute the SE and CI for these estimates.

In [ ]:
iters = 1000
estimates = np.array([compstats.resample_rows(live).totalwgt_lb.mean() for _ in range(iters)])
summarize(estimates)

## Visualizing uncertainty

To show the uncertainty of the estimated slope and intercept, we can generate a fitted line for each resampled estimate and plot them on top of each other.

In [ ]:
for slope, inter in zip(slopes, inters):
    fxs, fys = fit_line(age_means, inter, slope)
    plt.plot(fxs, fys, color='darkgray', alpha=0.01)
    plt.plot(fxs, fys, color='gray', alpha=0.01)
plt.xlabel("Mother's age (years)")
plt.ylabel("Residual (lbs)")
plt.xlim([10, 45]);

Or we can make a neater (and more efficient plot) by computing fitted lines and finding percentiles of the fits for each value of the dependent variable.

In [ ]:
def plot_confidence_intervals(xs, inters, slopes, percent=90, **options):
    fys_seq = []
    for inter, slope in zip(inters, slopes):
        fxs, fys = fit_line(xs, inter, slope)
        fys_seq.append(fys)

    p = (100 - percent) / 2
    percents = p, 100 - p
    low, high = compstats.percentile_rows(fys_seq, percents)
    plt.fill_between(
        fxs, low, high, **options
    )

This example shows the confidence interval for the fitted values at each mother's age.

In [ ]:
plot_confidence_intervals(
    age_means, inters, slopes,
    percent=50,
    color='gray',
    alpha=0.5,
    label='50% CI'
)

plot_confidence_intervals(
    age_means, inters, slopes,
    percent=90, 
    color='gray',
    alpha=0.3,
    label='90% CI'
)

plt.xlabel("Mother's age (years)")
plt.ylabel("Residual (lbs)")
plt.legend(loc='upper left')
plt.xlim([13, 45]);

50% and 90% confidence intervals showing variability in thefitted line due to sampling error of intercept and slope.

## Goodness of fit

There are several ways to measure the quality of a linear model, or goodness of fit. One of the simplest is the standard deviation of the residuals.

If you use a linear model to make predictions, `std(res)` is the root mean squared error (RMSE) of your predictions. For example, if you use mother’s age to guess birth weight, the RMSE of your guess would be 1.40 lbs.

In [ ]:
np.round(compstats.std(live.residual), 2)

If you guess birth weight without knowing the mother’s age, the RMSE of your guess is `std(ys)`, which is 1.41 lbs. So in this example, knowing a mother’s age does not improve the predictions substantially.

In [ ]:
np.round(compstats.std(weights), 2)

Another way to measure goodness of fit is the coefficient of determination, usually denoted $R^2$ and called “R-squared”:

In [ ]:
def coef_determination(ys: np.array, res: np.array):
    # the variance explained by the model (1-res(var)) as a proportion of the total variance
    return 1 - res.var() / ys.var()

`var(res)` is the MSE of your guesses using the model, `var(ys)` is the MSE without it. So their ratio is the fraction of MSE that remains if you use the model, and $R^2$ is the fraction of MSE the model eliminates.

For birth weight and mother's age $R^2$ is very small, indicating that the mother's age predicts a small part of the variance in birth weight.

In [ ]:
inter, slope = least_squares(ages, weights)
res = residuals(ages, weights, inter, slope)
r2 = coef_determination(weights, res)
np.round(r2, 4)

We can confirm that $R^2 = \rho^2$ (Pearson’s coefficient of correlation):

In [ ]:
print(f'rho: {compstats.corr(ages, weights):0.4f}, R: {np.sqrt(r2):0.4f}')

To express predictive power, I think it's useful to compare the standard deviation of the residuals to the standard deviation of the dependent variable, as a measure RMSE if you try to guess birth weight with and without taking into account mother's age.

In [ ]:
print(f'std(ys): {compstats.std(weights):0.3f}, std(res): {compstats.std(res):0.3f}')

For example, when people talk about the validity of the SAT (a standardized test used for college admission in the U.S.) they often talk about correlations between SAT scores and other measures of intelligence.

According to one study, there is a Pearson correlation of $\rho = 0.72$ between total SAT scores and IQ scores, which sounds like a strong correlation, but $R^2=\rho^2=0.52$ so SAT scores account for only 52% of the variance in IQ.

IQ scores are normalized with `std(ys) = 15`, so

In [ ]:
var_ys = 15**2
rho = 0.72
r2 = rho**2
# coefficient of determination
var_res = (1 - r2) * var_ys
std_res = np.sqrt(var_res)
np.round(std_res, 2)

So using SAT score to predict IQ reduces RMSE from 15 points to 10.4 points. A correlation of 0.72 yields a reduction in RMSE of only 31%.

If you see a correlation that looks impressive, remember that $R^2$ is a better indicator of reduction in MSE, and reduction in RMSE is a better indicator of predictive power.

## Hypothesis testing with slopes

Another approach is to test whether the apparent slope is due to chance. The null hypothesis is that the slope is actually zero; in that case we can model the birth weights as random variations around their mean. Here’s a hypothesis test for this model:

In [ ]:
class SlopeTest:
    
    def __init__(self, data: hypothesis.GroupPair):
        # ages, weights
        weights = data.group2
        self.ybar = weights.mean()
        self.res = weights - self.ybar

    def test_statistic(self, data: hypothesis.GroupPair):
        # the slope estimated by least squares
        _, slope = least_squares(data.group1, data.group2)
        return slope

    def resample(self, data: hypothesis.GroupPair):
        # permute the deviations (residuals) and add them to the mean
        weights = self.ybar + np.random.permutation(self.res)
        return hypothesis.GroupPair(data.group1, weights)

The data are represented as sequences of ages and weights. The test statistic is the slope estimated by `least_squares`. The model of the null hypothesis is represented by the mean weight of all babies and the deviations from the mean. To generate simulated data, we permute the deviations and add them to the mean.

And it is.

In [ ]:
data = hypothesis.GroupPair(ages, weights)
st = SlopeTest(data)
actual = st.test_statistic(data)
print(actual)

In [ ]:
estimates = hypothesis.run_model(
    data,
    test_stat=st.test_statistic,
    sampler=st.resample
)
print(f'actual: {actual:0.2}, \
    p-value {hypothesis.p_value(estimates, actual)}, \
    max: {np.max(estimates):0.4}')

In [ ]:
sns.histplot(x=estimates);
plt.axvline(actual, color='darkred', linestyle='--');

Under the null hypothesis, the largest slope we observe after 1000 tries is substantially less than the observed value.

We can also use resampling to estimate the sampling distribution of the slope.

The distribution of slopes under the null hypothesis, and the sampling distribution of the slope under resampling, have the same shape, but one has mean at 0 and the other has mean at the observed slope.

To compute a p-value, we can count how often the estimated slope under the null hypothesis exceeds the observed slope, or how often the estimated slope under resampling falls below 0.

In [ ]:
sns.ecdfplot(x=estimates, label='Null hypothesis');
sns.ecdfplot(x=slopes, label='Sampling distribution');
plt.axvline(estimates.mean(), color='gray', linestyle='--')
plt.axvline(slopes.mean(), color='gray', linestyle='--')
plt.xlim([-0.03, 0.03])
plt.xlabel('slope (lbs / year)')
plt.ylabel('CDF')
plt.legend(loc='upper left');

The sampling distribution of the estimated slope and the distribution of slopes generated under the null hypothesis. The vertical lines are at 0 and the observed slope, 0.017 lbs/year.

Here's how to get a p-value from the sampling distribution.

In [ ]:
sampling_cdf = Cdf.from_seq(slopes)

In [ ]:
# proportion of slopes that match the null distribution (i.e zero or no slope)
pvalue = sampling_cdf.prob(0)
pvalue

## Weighted resampling

So far we have treated the NSFG data as if it were a representative sample, but as I mentioned in Section 1.2, it is not. The survey deliberately oversamples several groups in order to improve the chance of getting statistically significant results; that is, in order to improve the power of tests involving these groups.

This survey design is useful for many purposes, but it means that we cannot use the sample to estimate values for the general population without accounting for the sampling process.

For each respondent, the NSFG data includes a variable called finalwgt, which is the number of people in the general population the respondent represents. This value is called a sampling weight.

Resampling provides a convenient way to take into account the sampling weights associated with respondents in a stratified survey design.

As an example, if you survey 100,000 people in a country of 300 million, each respondent represents 3,000 people. If you oversample one group by a factor of 2, each person in the oversampled group would have a lower weight, about 1500.

To correct for oversampling, we can use resampling; that is, we can draw samples from the survey using probabilities proportional to sampling weights. Then, for any quantity we want to estimate, we can generate sampling dis- tributions, standard errors, and confidence intervals. As an example, I will estimate mean birth weight with and without sampling weights.

Earlier, we saw `resample_rows`, which chooses rows from a DataFrame, giving each row the same probability. 

Now we need to do the same thing using probabilities proportional to sampling weights. 

`resample_rows_weighted` takes a DataFrame, resamples rows according to the weights in `finalwgt`, and returns a DataFrame containing the resampled rows:

In [ ]:
cdf = Cdf.from_dict(dict(live.finalwgt))

In [ ]:
# some rows appear far more than others
pd.Series(cdf.sample(len(live))).value_counts()

In [ ]:
def resample_rows_weighted(df, column='finalwgt'):
    weights = df[column]
    cdf = Cdf.from_dict(dict(weights))
    indices = cdf.sample(len(weights))
    return df.loc[indices]

We can use it to estimate the mean birthweight and compute SE and CI.

In [ ]:
iters = 100
estimates = np.array([
    resample_rows_weighted(live).totalwgt_lb.mean() for _ in range(iters)
])
summarize(estimates)

And here's what the same calculation looks like if we ignore the weights.

In [ ]:
estimates = np.array([
    compstats.resample_rows(live).totalwgt_lb.mean() for _ in range(iters)
])
summarize(estimates)

In this example, the effect of weighting is small but non-negligible. The difference in estimated means, with and without weighting, is about 0.08 pounds, or 1.3 ounces. This difference is substantially larger than the standard error of the estimate, 0.014 pounds, which implies that the difference is not due to chance.

# Exercises

**Exercise:** Using the data from the BRFSS, compute the linear least squares fit for log(weight) versus height. How would you best present the estimated parameters for a model like this where one of the variables is log-transformed? If you were trying to guess someone’s weight, how much would it help to know their height?

Like the NSFG, the BRFSS oversamples some groups and provides a sampling weight for each respondent. In the BRFSS data, the variable name for these weights is totalwt. Use resampling, with and without weights, to estimate the mean height of respondents in the BRFSS, the standard error of the mean, and a 90% confidence interval. How much does correct weighting affect the estimates?

Read the BRFSS data and extract heights and log weights.

In [ ]:
df = brfss.read_brfss().dropna(subset=['weight', 'height'])

In [ ]:
df.apply(lambda col: col.isna().sum())

In [ ]:
df['log_weight'] = np.log10(df.weight)

Estimate intercept and slope.

In [ ]:
# Solution
inter, slope = least_squares(df.height, df.log_weight)
np.round((inter, slope,), 4)

Make a scatter plot of the data and show the fitted line.

In [ ]:
inter + 75 * slope

In [ ]:
sns.scatterplot(
    data=df,
    x='height',
    y='log_weight',
    alpha=0.01,
    s=5
)
plt.axline(
    (75, inter + 75*slope),
    slope=slope,
    color='darkred', linestyle='--')
plt.xlabel('Height (cm)')
plt.ylabel('log10 weight (kg)');

Make the same plot but apply the inverse transform to show weights on a linear (not log) scale.

In [ ]:
fxs, fys = fit_line(df.height, inter, slope)

In [ ]:
sns.scatterplot(
    data=df,
    x='height',
    y='weight',
    alpha=0.01,
    s=5
)
plt.plot(fxs, 10**fys, color='darkred', linestyle='--')
plt.xlabel('Height (cm)')
plt.ylabel('log10 weight (kg)');

Plot percentiles of the residuals.

In [ ]:
df['residual'] = residuals(df.height, df.log_weight, inter, slope)

In [ ]:
bins = np.arange(130, 210, 5)
indices = np.digitize(df.height, bins)
groups = df.groupby(indices)

means = [group.height.mean() for i, group in groups][1:-1]
cdfs = [Cdf.from_seq(group.residual) for i, group in groups][1:-1]

In [ ]:
for percent in [75, 50, 25]:
    ys = [cdf.percentile(percent) for cdf in cdfs]
    plt.plot(means, ys, label=f'{percent:d}th')
plt.xlabel('Height (cm)')
plt.ylabel('residual weight (kg)');
plt.legend(loc='upper right');

Compute correlation.

In [ ]:
# Solution

rho = compstats.corr(df.height, df.log_weight)
np.round(rho, 2)

Compute coefficient of determination.

In [ ]:
# Solution

r2 = coef_determination(df.log_weight, df.residual)
r2

Confirm that $R^2 = \rho^2$.

In [ ]:
# Solution

np.round(rho**2 - r2, 2)

Compute `std(ys)`, which is the RMSE of predictions that don't use height.

In [ ]:
# Solution

std_ys = compstats.std(df.log_weight)
std_ys

Compute std(res), the RMSE of predictions that do use height.

In [ ]:
# Solution

std_res = compstats.std(df.residual)
std_res

How much does height information reduce RMSE?

In [ ]:
# Solution

1 - std_res / std_ys

Use resampling to compute sampling distributions for inter and slope.

In [ ]:
# Solution

t = []
for _ in range(100):
    sample = compstats.resample_rows(df)
    estimates = least_squares(sample.height, np.log10(sample.weight))
    t.append(estimates)

inters, slopes = zip(*t)

Plot the sampling distribution of slope.

In [ ]:
sns.ecdfplot(
    x = slopes
);

Compute the p-value of the slope.

In [ ]:
# Solution
cdf = Cdf.from_seq(slopes)
pvalue = cdf.prob(0)
pvalue

Compute the 90% confidence interval of slope.

In [ ]:
np.round(cdf.values((0.05, 0.95)), 4)

Compute the mean of the sampling distribution.

In [ ]:
# Solution

mean = np.mean(slopes)
mean

Compute the standard deviation of the sampling distribution, which is the standard error.

In [ ]:
# Solution

stderr = compstats.std(np.array(slopes))
stderr

Resample rows without weights, compute mean height, and summarize results.

In [ ]:
estimates_unweighted = np.array([
    compstats.resample_rows(df).height.mean() for _ in range(100)
])
summarize(estimates_unweighted)

Resample rows with weights.  Note that the weight column in this dataset is called `finalwt`.

In [ ]:
# this takes a while
estimates_weighted = np.array([
    resample_rows_weighted(df, 'finalwt').height.mean() for _ in range(100)
])
summarize(estimates_weighted)

The estimated mean height is almost 2 cm taller if we take into account the sampling weights, and this difference is much bigger than the sampling error.